In [1]:
load.ivy("joda-time" % "joda-time" % "2.8")

:: problems summary ::
:::: WARNINGS
	Unable to reparse com.github.alexarchambault.jupyter#jupyter-scala-api_2.11.6;0.2.0-SNAPSHOT from sonatype-snapshots, using Fri Jun 05 10:12:58 CEST 2015

	Choosing sonatype-snapshots for com.github.alexarchambault.jupyter#jupyter-scala-api_2.11.6;0.2.0-SNAPSHOT

	Unable to reparse com.github.alexarchambault#ammonite-api_2.11.6;0.3.1-SNAPSHOT from sonatype-snapshots, using Thu Sep 10 11:11:45 CEST 2015

	Choosing sonatype-snapshots for com.github.alexarchambault#ammonite-api_2.11.6;0.3.1-SNAPSHOT

	Unable to reparse com.github.alexarchambault.jupyter#jupyter-api_2.11;0.2.0-SNAPSHOT from sonatype-snapshots, using Mon Jun 01 02:54:01 CEST 2015

	Choosing sonatype-snapshots for com.github.alexarchambault.jupyter#jupyter-api_2.11;0.2.0-SNAPSHOT



In [2]:
import scala.io.Source
import java.io.{PrintWriter, File}
import org.joda.time.{DateTime, DateTimeZone, Duration, Interval}
import org.joda.time.format.DateTimeFormat
import scala.collection.immutable.TreeMap

import scala.io.Source
import java.io.{PrintWriter, File}
import org.joda.time.{DateTime, DateTimeZone, Duration, Interval}
import org.joda.time.format.DateTimeFormat
import scala.collection.immutable.TreeMap

In [3]:
val directory = new File("/Users/mg/Downloads")
val history_file = new File(directory, "mock.csv")
val headers = Source.fromFile(history_file).getLines.next split "," zipWithIndex
val dtf = DateTimeFormat.forPattern("yyyy-MM-dd HH:mm:ss").withZoneUTC

val NB_STEPS = 10

implicit def orderingByDateTime[A <: DateTime]: Ordering[A] = Ordering.by(d => d.getMillis)

case class Row(project: String, 
               file: String,
               jpa: Option[Interval], 
               jdbc: Option[Interval],
               hbm: Option[Interval])
{
    def is_entity_related(): Boolean = jpa.isDefined || jdbc.isDefined || hbm.isDefined
    
    def dates: Seq[DateTime] =
    {
        var ret = Seq[DateTime]()
        ret = if(jpa.isDefined)  ret ++ Seq(jpa.get.getStart,jpa.get.getEnd) else ret
        ret = if(jdbc.isDefined) ret ++ Seq(jdbc.get.getStart,jdbc.get.getEnd) else ret
        ret = if(hbm.isDefined)  ret ++ Seq(hbm.get.getStart,hbm.get.getEnd) else ret
        
        ret
    }
    
    def is_jpa(t: DateTime): Boolean = jpa match {
        case s: Some[Interval] => !((t isBefore s.get.getStart) || (t isAfter s.get.getEnd))
        case _ => false
    }
    
    def is_jdbc(t: DateTime): Boolean = jdbc match {
        case s: Some[Interval] => !((t isBefore s.get.getStart) || (t isAfter s.get.getEnd))
        case _ => false
    }
    
    def is_hbm(t: DateTime): Boolean = hbm match {
        case s: Some[Interval] => !((t isBefore s.get.getStart) || (t isAfter s.get.getEnd))
        case _ => false
    }
}

object Row
{
    def apply(line: String) = {
        val elements = line.split(",")
        val project = elements(0)
        val file = elements(1)
        val jpa = elements(2) match {
            case "NA" => None
            case ""   => None
            case _: String => Some(new Interval( dtf.parseDateTime(elements(2)) , dtf.parseDateTime(elements(3))))  
        }
        
        val jdbc = elements(4) match {
            case "NA" => None
            case ""   => None
            case _: String => Some(new Interval(dtf.parseDateTime(elements(4)) , dtf.parseDateTime(elements(5))))  
        }
        
        val hbm = elements(6) match {
            case "NA" => None
            case ""   => None
            case _: String => Some(new Interval(dtf.parseDateTime(elements(6)) , dtf.parseDateTime(elements(7))))  
        }
        
        
        new Row(project,file,jpa,jdbc,hbm)
    }
}

directory: File = /Users/mg/Downloads
history_file: File = /Users/mg/Downloads/mock.csv
headers: Array[(String, Int)] = Array(
  ("project", 0),
  ("file", 1),
  ("jpa_first", 2),
  ("jpa_last", 3),
  ("jdbc_first", 4),
  ("jdbc_last", 5),
  ("hbm_first", 6),
  ("hbm_last", 7)
)
dtf: org.joda.time.format.DateTimeFormatter = org.joda.time.format.DateTimeFormatter@1572dcad
NB_STEPS: Int = 10
defined function orderingByDateTime
defined class Row
defined object Row

In [4]:
val entries = Source.fromFile(history_file).getLines.drop(1) map (Row.apply) toList
val projects = entries.groupBy(_.project)


entries: List[Row] = List(
  Row(
    "projectA",
    "file1",
    Some(2010-01-01T00:00:00.000Z/2010-01-10T12:34:56.000Z),
    None,
    None
  ),
  Row(
    "projectA",
    "file2",
    Some(2010-01-01T00:00:00.000Z/2010-01-10T12:34:56.000Z),
    Some(2010-01-05T12:34:56.000Z/2010-01-17T12:34:56.000Z),
    None
  ),
...
projects: Map[String, List[Row]] = Map(
  "projectB" -> List(
    Row(
      "projectB",
      "file1",
      Some(2010-01-01T00:00:00.000Z/2010-01-10T12:34:56.000Z),
      None,
      None
    ),
    Row(
      "projectB",
      "file2",
      Some(2010-01-01T00:00:00.000Z/2010-07-10T12:34:56.000Z),
      None,
      None
...

In [5]:
// Writes on file the evolution (step by step) of technology coverage of in projects

val output = new PrintWriter(new File(directory, "techno-coverage.csv"))
output.println("project,step,entities,jpa,jdbc,hbm,jpa-jdbc,jpa-hbm,jdbc-hbm,jpa-jdbc-hbm")

projects.foreach(p => {
    val dates = p._2.flatMap(r => r.dates)
    val min_date = dates.minBy(d => d.getMillis)
    val max_date = dates.maxBy(d => d.getMillis)
    val interval = new Duration(min_date,max_date).getStandardSeconds / NB_STEPS
    
    // Divide project history into regularly spaced datetime
    val steps = (0 to NB_STEPS) map (s => s -> (min_date plus Duration.standardSeconds(s*interval)))
    
    steps.foreach(step => {
        val contains_jpa  = p._2 filter (r => r.is_jpa(step._2)) toSet
        val contains_jdbc = p._2 filter (r => r.is_jdbc(step._2)) toSet
        val contains_hbm  = p._2 filter (r => r.is_hbm(step._2)) toSet
        
        val nb_entities = (contains_jpa union contains_jdbc union contains_hbm) size
        
        val nb_jpa  = ((contains_jpa diff contains_jdbc) diff contains_hbm) size
        val nb_jdbc = ((contains_jdbc diff contains_jpa) diff contains_hbm) size
        val nb_hbm  = ((contains_hbm diff contains_jpa) diff contains_jdbc) size
        
        val nb_jpa_jdbc = ((contains_jpa intersect contains_jdbc) diff contains_hbm) size
        val nb_jpa_hbm  = ((contains_jpa intersect contains_hbm) diff contains_jdbc) size
        val nb_jdbc_hbm = ((contains_jdbc intersect contains_hbm) diff contains_jpa) size
        
        val nb_jpa_jdbc_hbm = (contains_jpa intersect contains_jdbc intersect contains_hbm) size
        
        assert(nb_entities == (
            nb_jpa + nb_jdbc + nb_hbm +
            nb_jpa_jdbc + nb_jpa_hbm + nb_jdbc_hbm +
            nb_jpa_jdbc_hbm
        ))
        
        output.println(p._1 + "," + step._1.toFloat / NB_STEPS + "," + nb_entities + "," + 
                       nb_jpa.toFloat / math.max(nb_entities,1) + "," + 
                       nb_jdbc.toFloat / math.max(nb_entities,1) + "," + 
                       nb_hbm.toFloat / math.max(nb_entities,1) + "," + 
                       nb_jpa_jdbc.toFloat / math.max(nb_entities,1) + "," + 
                       nb_jpa_hbm.toFloat / math.max(nb_entities,1) + "," + 
                       nb_jdbc_hbm.toFloat / math.max(nb_entities,1) + "," + 
                       nb_jpa_jdbc_hbm.toFloat / math.max(nb_entities,1))
    })
    
})

output.close

output: PrintWriter = java.io.PrintWriter@6d4a0e80

In [17]:
// Writes on file a flat representation of the evolution (step by step) of technology coverage in projects

val output = new PrintWriter(new File(directory, "flat-techno-coverage.csv"))

output.println("project," + (0 to NB_STEPS).map (s => "entities-" + s + 
                                                      ",jpa-" + s +
                                                      ",jdbc-" + s +
                                                      ",hbm-" + s +
                                                      ",jpa-jdbc-" + s +
                                                      ",jpa-hbm-" + s +
                                                      ",jdbc-hbm-" + s +
                                                      ",jpa-jdbc-hbm-" + s
                                                ).mkString(","))

projects.foreach(p => {
    val dates = p._2.flatMap(r => r.dates)
    val min_date = dates.minBy(d => d.getMillis)
    val max_date = dates.maxBy(d => d.getMillis)
    val interval = new Duration(min_date,max_date).getStandardSeconds / NB_STEPS
    
    output.print(p._1 + ",")
    
    // Divide project history into regularly spaced datetime
    val steps = (0 to NB_STEPS) map (s => s -> (min_date plus Duration.standardSeconds(s*interval)))
    
    output.println(steps.map(step => {
        val contains_jpa  = p._2 filter (r => r.is_jpa(step._2)) toSet
        val contains_jdbc = p._2 filter (r => r.is_jdbc(step._2)) toSet
        val contains_hbm  = p._2 filter (r => r.is_hbm(step._2)) toSet
        
        val nb_entities = (contains_jpa union contains_jdbc union contains_hbm) size
        
        val nb_jpa  = ((contains_jpa diff contains_jdbc) diff contains_hbm) size
        val nb_jdbc = ((contains_jdbc diff contains_jpa) diff contains_hbm) size
        val nb_hbm  = ((contains_hbm diff contains_jpa) diff contains_jdbc) size
        
        val nb_jpa_jdbc = ((contains_jpa intersect contains_jdbc) diff contains_hbm) size
        val nb_jpa_hbm  = ((contains_jpa intersect contains_hbm) diff contains_jdbc) size
        val nb_jdbc_hbm = ((contains_jdbc intersect contains_hbm) diff contains_jpa) size
        
        val nb_jpa_jdbc_hbm = (contains_jpa intersect contains_jdbc intersect contains_hbm) size
        
        assert(nb_entities == (
            nb_jpa + nb_jdbc + nb_hbm +
            nb_jpa_jdbc + nb_jpa_hbm + nb_jdbc_hbm +
            nb_jpa_jdbc_hbm
        ))
        
        (nb_entities + "," + 
                nb_jpa.toFloat / math.max(nb_entities,1) + "," + 
                nb_jdbc.toFloat / math.max(nb_entities,1) + "," + 
                nb_hbm.toFloat / math.max(nb_entities,1) + "," + 
                nb_jpa_jdbc.toFloat / math.max(nb_entities,1) + "," + 
                nb_jpa_hbm.toFloat / math.max(nb_entities,1) + "," + 
                nb_jdbc_hbm.toFloat / math.max(nb_entities,1) + "," + 
                nb_jpa_jdbc_hbm.toFloat / math.max(nb_entities,1))
    }).mkString(","))
    
})


output.close


output: PrintWriter = java.io.PrintWriter@7802b9be